# Case Study 7: Scalability Analysis - Computational Time & Performance

This notebook conducts a formal quantitative scalability analysis by executing simulations with increasing agent populations to assess computational time and analyze the impact of agent count on market KPIs.

## 🎯 Objectives

1. **Computational Time Analysis**: Measure training time across different agent populations
2. **Training Paradigm Comparison**: Compare CTCE, CTDE, and DTDE training modes
3. **Algorithm Comparison**: Evaluate PPO, APPO, and SAC algorithms
4. **KPI Impact Analysis**: Assess how agent count affects market performance metrics
5. **Scalability Limits**: Identify computational bottlenecks and scalability thresholds

## 📋 Table of Contents

1. [Setup & Imports](#setup--imports)
2. [Configuration](#configuration)
3. [Agent Population Scenarios](#agent-population-scenarios)
4. [Scalability Test Execution](#scalability-test-execution)
5. [Computational Time Analysis](#computational-time-analysis)
6. [KPI Impact Analysis](#kpi-impact-analysis)
7. [Results Summary](#results-summary)

---


## 🛠️ Setup & Imports


In [ ]:
# Standard library imports
import sys
import os
import warnings
import time
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"📁 Project root: {project_root}")


In [ ]:
# Import project-specific modules
try:
    from src.agent.battery import Battery
    from src.agent.der import DERAgent
    from src.grid.network import GridNetwork, GridTopology
    from src.market.matching import MarketConfig
    from src.market.mechanism import ClearingMechanism
    from src.profile.der import DERProfileHandler
    from src.profile.dso import DSOProfileHandler
    from src.environment.train import RLTrainer, TrainingMode, RLAlgorithm
    
    print("✅ Project modules imported successfully!")
    
    # Display available training modes and algorithms
    print("\n📋 Available Training Modes:")
    for mode in TrainingMode:
        print(f"  - {mode.name}: {mode.value}")
    
    print("\n📋 Available Algorithms:")
    for algo in RLAlgorithm:
        print(f"  - {algo.name}: {algo.value}")
        
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Please ensure you're running this notebook from the correct directory")
    print("and that all dependencies are installed.")


## ⚙️ Configuration


In [ ]:
@dataclass
class ScalabilityConfig:
    """Configuration for scalability analysis."""
    
    # Agent population sizes to test
    AGENT_POPULATIONS = [1, 5, 10, 20, 50, 100]  # Can add more: 200, 500, 1000
    
    # Training parameters (reduced for scalability testing)
    TRAINING_EPISODES = 50  # Reduced for faster scalability testing
    EVALUATION_EPISODES = 10
    MAX_STEPS = 24  # 24-hour simulation
    
    # Grid configuration
    GRID_CAPACITY = 5000.0  # kW (scaled for larger populations)
    GRID_TOPOLOGY = GridTopology.MESH  # Mesh topology for scalability
    
    # Market parameters
    MIN_PRICE = 40.0  # $/MWh
    MAX_PRICE = 240.0  # $/MWh
    MIN_QUANTITY = 0.1  # kWh
    MAX_QUANTITY = 200.0  # kWh
    
    # Training modes and algorithms to test
    TRAINING_MODES = [TrainingMode.CTCE, TrainingMode.CTDE, TrainingMode.DTDE]
    ALGORITHMS = [RLAlgorithm.PPO, RLAlgorithm.APPO, RLAlgorithm.SAC]
    
    # Results storage
    RESULTS_DIR = Path("scalability_results")
    RESULTS_DIR.mkdir(exist_ok=True)

# Display configuration
print("📊 Scalability Analysis Configuration:")
print("=" * 80)
print(f"  Agent Populations: {ScalabilityConfig.AGENT_POPULATIONS}")
print(f"  Training Episodes: {ScalabilityConfig.TRAINING_EPISODES}")
print(f"  Evaluation Episodes: {ScalabilityConfig.EVALUATION_EPISODES}")
print(f"  Grid Capacity: {ScalabilityConfig.GRID_CAPACITY} kW")
print(f"  Grid Topology: {ScalabilityConfig.GRID_TOPOLOGY.value}")
print(f"  Training Modes: {[m.name for m in ScalabilityConfig.TRAINING_MODES]}")
print(f"  Algorithms: {[a.name for a in ScalabilityConfig.ALGORITHMS]}")
print(f"  Total Test Combinations: {len(ScalabilityConfig.AGENT_POPULATIONS) * len(ScalabilityConfig.TRAINING_MODES) * len(ScalabilityConfig.ALGORITHMS)}")
print("=" * 80)


## 👥 Agent Population Scenarios


In [ ]:
def create_agents_for_population(num_agents: int, seed: int = 42) -> List[DERAgent]:
    """Create a population of agents for scalability testing.
    
    Args:
        num_agents: Number of agents to create
        seed: Random seed for reproducibility
        
    Returns:
        List of DERAgent objects
    """
    np.random.seed(seed)
    agents = []
    profile_handler = DERProfileHandler()
    
    print(f"  Creating {num_agents} agents...")
    
    for i in range(num_agents):
        # Vary agent capacities for diversity
        capacity = np.random.uniform(50.0, 150.0)
        battery_capacity = capacity * np.random.uniform(0.5, 1.0)
        
        # Generate profiles
        generation, demand = profile_handler.get_energy_profiles(
            ScalabilityConfig.MAX_STEPS,
            capacity
        )
        
        agent = DERAgent(
            id=f"agent_{i:04d}",
            capacity=capacity,
            battery=Battery(
                nominal_capacity=battery_capacity,
                min_soc=0.1,
                max_soc=0.9,
                charge_efficiency=0.95,
                discharge_efficiency=0.95
            ),
            generation_profile=generation,
            demand_profile=demand
        )
        agents.append(agent)
    
    return agents

def create_scenario_config(num_agents: int) -> Dict[str, Any]:
    """Create environment configuration for a given number of agents.
    
    Args:
        num_agents: Number of agents in the scenario
        
    Returns:
        Environment configuration dictionary
    """
    agents = create_agents_for_population(num_agents)
    
    grid_network = GridNetwork(
        topology=ScalabilityConfig.GRID_TOPOLOGY,
        num_nodes=num_agents,
        capacity=ScalabilityConfig.GRID_CAPACITY,
        seed=42
    )
    
    grid_network.assign_agents_to_graph(agents)
    
    market_config = MarketConfig(
        min_price=ScalabilityConfig.MIN_PRICE,
        max_price=ScalabilityConfig.MAX_PRICE,
        min_quantity=ScalabilityConfig.MIN_QUANTITY,
        max_quantity=ScalabilityConfig.MAX_QUANTITY,
        price_mechanism=ClearingMechanism.AVERAGE,
        enable_partner_preference=False,
        blockchain_difficulty=1,
        visualize_blockchain=False
    )
    
    der_profile_handler = DERProfileHandler()
    dso_profile_handler = DSOProfileHandler(
        min_price=ScalabilityConfig.MIN_PRICE,
        max_price=ScalabilityConfig.MAX_PRICE
    )
    
    return {
        "max_steps": ScalabilityConfig.MAX_STEPS,
        "agents": agents,
        "market_config": market_config,
        "grid_network": grid_network,
        "der_profile_handler": der_profile_handler,
        "dso_profile_handler": dso_profile_handler,
        "enable_reset_dso_profiles": True,
        "enable_asynchronous_order": True,
        "max_error": 0.15,
        "num_anchor": 8,
        "seed": 42
    }

# Test agent creation
print("🧪 Testing agent creation:")
test_agents = create_agents_for_population(5)
print(f"✅ Created {len(test_agents)} test agents successfully!")


In [ ]:
def run_scalability_test(num_agents: int, 
                        training_mode: TrainingMode,
                        algorithm: RLAlgorithm,
                        verbose: bool = True) -> Dict[str, Any]:
    """Run a single scalability test and measure computational time.
    
    Args:
        num_agents: Number of agents
        training_mode: Training mode (CTCE, CTDE, DTDE)
        algorithm: RL algorithm (PPO, APPO, SAC)
        verbose: Whether to print progress
        
    Returns:
        Dictionary with test results including timing and KPIs
    """
    if verbose:
        print(f"\n🔄 Testing: {num_agents} agents, {training_mode.name}, {algorithm.name}")
    
    # Create scenario configuration
    env_config = create_scenario_config(num_agents)
    
    # Measure training time
    start_time = time.time()
    
    try:
        # Create trainer
        trainer = RLTrainer(
            env_config=env_config,
            algorithm=algorithm,
            training=training_mode,
            iters=ScalabilityConfig.TRAINING_EPISODES,
            cpus=1,
            gpus=0
        )
        
        # Train
        trainer.train()
        
        training_time = time.time() - start_time
        
        # Calculate time per agent and per episode
        time_per_agent = training_time / num_agents if num_agents > 0 else 0
        time_per_episode = training_time / ScalabilityConfig.TRAINING_EPISODES if ScalabilityConfig.TRAINING_EPISODES > 0 else 0
        
        if verbose:
            print(f"  ✅ Completed in {training_time:.2f}s ({time_per_agent:.3f}s/agent, {time_per_episode:.3f}s/episode)")
        
        return {
            "num_agents": num_agents,
            "training_mode": training_mode.name,
            "algorithm": algorithm.name,
            "training_time": training_time,
            "time_per_agent": time_per_agent,
            "time_per_episode": time_per_episode,
            "status": "success",
            "trainer": trainer
        }
        
    except Exception as e:
        training_time = time.time() - start_time
        if verbose:
            print(f"  ❌ Failed after {training_time:.2f}s: {str(e)}")
        
        return {
            "num_agents": num_agents,
            "training_mode": training_mode.name,
            "algorithm": algorithm.name,
            "training_time": training_time,
            "time_per_agent": 0,
            "time_per_episode": 0,
            "status": "failed",
            "error": str(e)
        }

# Run a quick test
print("🧪 Running quick scalability test (1 agent, PPO, CTDE)...")
test_result = run_scalability_test(1, TrainingMode.CTDE, RLAlgorithm.PPO, verbose=True)
print(f"\n✅ Test completed: {test_result['status']}")


In [ ]:
# Run full scalability analysis
print("🚀 Starting Full Scalability Analysis")
print("=" * 80)
print(f"Total test combinations: {len(ScalabilityConfig.AGENT_POPULATIONS) * len(ScalabilityConfig.TRAINING_MODES) * len(ScalabilityConfig.ALGORITHMS)}")
print(f"Estimated time: ~{len(ScalabilityConfig.AGENT_POPULATIONS) * len(ScalabilityConfig.TRAINING_MODES) * len(ScalabilityConfig.ALGORITHMS) * 2} minutes (rough estimate)")
print("=" * 80)

results = []
total_tests = len(ScalabilityConfig.AGENT_POPULATIONS) * len(ScalabilityConfig.TRAINING_MODES) * len(ScalabilityConfig.ALGORITHMS)
current_test = 0

for num_agents in ScalabilityConfig.AGENT_POPULATIONS:
    for training_mode in ScalabilityConfig.TRAINING_MODES:
        for algorithm in ScalabilityConfig.ALGORITHMS:
            current_test += 1
            print(f"\n[{current_test}/{total_tests}] Testing {num_agents} agents, {training_mode.name}, {algorithm.name}")
            
            result = run_scalability_test(num_agents, training_mode, algorithm, verbose=False)
            results.append(result)
            
            # Save intermediate results
            if current_test % 5 == 0:
                df_temp = pd.DataFrame(results)
                df_temp.to_csv(ScalabilityConfig.RESULTS_DIR / "scalability_results_intermediate.csv", index=False)
                print(f"  💾 Intermediate results saved")

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Save final results
df_results.to_csv(ScalabilityConfig.RESULTS_DIR / "scalability_results_final.csv", index=False)

print("\n" + "=" * 80)
print("✅ Scalability Analysis Complete!")
print(f"Total tests: {len(results)}")
print(f"Successful: {sum(1 for r in results if r['status'] == 'success')}")
print(f"Failed: {sum(1 for r in results if r['status'] == 'failed')}")
print(f"Results saved to: {ScalabilityConfig.RESULTS_DIR}")


## ⏱️ Computational Time Analysis


In [ ]:
# Load results if not already in memory
if 'df_results' not in locals() or df_results.empty:
    try:
        df_results = pd.read_csv(ScalabilityConfig.RESULTS_DIR / "scalability_results_final.csv")
        print("✅ Loaded results from file")
    except FileNotFoundError:
        print("❌ No results file found. Please run the scalability tests first.")
        df_results = pd.DataFrame()

if not df_results.empty:
    # Filter successful tests only
    df_success = df_results[df_results['status'] == 'success'].copy()
    
    print(f"\n📊 Results Summary:")
    print(f"  Total tests: {len(df_results)}")
    print(f"  Successful: {len(df_success)}")
    print(f"  Failed: {len(df_results) - len(df_success)}")
    
    if len(df_success) > 0:
        print(f"\n⏱️ Computational Time Statistics:")
        print(df_success.groupby(['training_mode', 'algorithm'])['training_time'].describe())
        
        print(f"\n📈 Time per Agent Statistics:")
        print(df_success.groupby(['training_mode', 'algorithm'])['time_per_agent'].describe())


In [ ]:
# Create computational time visualizations
if not df_results.empty and len(df_success) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Scalability Analysis: Computational Time', fontsize=16, fontweight='bold')
    
    # Plot 1: Training time vs number of agents (by training mode)
    ax1 = axes[0, 0]
    for mode in ScalabilityConfig.TRAINING_MODES:
        mode_data = df_success[df_success['training_mode'] == mode.name]
        if not mode_data.empty:
            grouped = mode_data.groupby('num_agents')['training_time'].mean()
            ax1.plot(grouped.index, grouped.values, marker='o', label=mode.name, linewidth=2)
    ax1.set_xlabel('Number of Agents')
    ax1.set_ylabel('Training Time (seconds)')
    ax1.set_title('Training Time vs Agent Population (by Training Mode)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    # Plot 2: Training time vs number of agents (by algorithm)
    ax2 = axes[0, 1]
    for algo in ScalabilityConfig.ALGORITHMS:
        algo_data = df_success[df_success['algorithm'] == algo.name]
        if not algo_data.empty:
            grouped = algo_data.groupby('num_agents')['training_time'].mean()
            ax2.plot(grouped.index, grouped.values, marker='s', label=algo.name, linewidth=2)
    ax2.set_xlabel('Number of Agents')
    ax2.set_ylabel('Training Time (seconds)')
    ax2.set_title('Training Time vs Agent Population (by Algorithm)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    
    # Plot 3: Time per agent vs number of agents
    ax3 = axes[1, 0]
    for mode in ScalabilityConfig.TRAINING_MODES:
        mode_data = df_success[df_success['training_mode'] == mode.name]
        if not mode_data.empty:
            grouped = mode_data.groupby('num_agents')['time_per_agent'].mean()
            ax3.plot(grouped.index, grouped.values, marker='o', label=mode.name, linewidth=2)
    ax3.set_xlabel('Number of Agents')
    ax3.set_ylabel('Time per Agent (seconds)')
    ax3.set_title('Time per Agent vs Agent Population')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Heatmap of training time (agents × training_mode × algorithm)
    ax4 = axes[1, 1]
    # Create pivot table for heatmap
    pivot_data = df_success.pivot_table(
        values='training_time',
        index='num_agents',
        columns=['training_mode', 'algorithm'],
        aggfunc='mean'
    )
    if not pivot_data.empty:
        sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='YlOrRd', ax=ax4, cbar_kws={'label': 'Time (s)'})
        ax4.set_title('Training Time Heatmap')
        ax4.set_xlabel('Training Mode × Algorithm')
        ax4.set_ylabel('Number of Agents')
    
    plt.tight_layout()
    plt.savefig(ScalabilityConfig.RESULTS_DIR / 'computational_time_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Computational time visualizations created and saved!")


In [ ]:
# Analyze scalability trends
if not df_results.empty and len(df_success) > 0:
    print("\n📊 Scalability Trends Analysis:")
    print("=" * 80)
    
    # Calculate scaling factors
    for mode in ScalabilityConfig.TRAINING_MODES:
        for algo in ScalabilityConfig.ALGORITHMS:
            combo_data = df_success[
                (df_success['training_mode'] == mode.name) & 
                (df_success['algorithm'] == algo.name)
            ]
            
            if len(combo_data) >= 2:
                # Calculate scaling factor (time increase per agent increase)
                sorted_data = combo_data.sort_values('num_agents')
                
                if len(sorted_data) > 1:
                    first_time = sorted_data.iloc[0]['training_time']
                    first_agents = sorted_data.iloc[0]['num_agents']
                    last_time = sorted_data.iloc[-1]['training_time']
                    last_agents = sorted_data.iloc[-1]['num_agents']
                    
                    if first_agents > 0 and first_time > 0:
                        scaling_factor = (last_time / first_time) / (last_agents / first_agents)
                        
                        print(f"\n{mode.name} + {algo.name}:")
                        print(f"  Agents: {first_agents} → {last_agents} ({last_agents/first_agents:.1f}x)")
                        print(f"  Time: {first_time:.2f}s → {last_time:.2f}s ({last_time/first_time:.1f}x)")
                        print(f"  Scaling Factor: {scaling_factor:.2f} (1.0 = linear, <1.0 = sub-linear, >1.0 = super-linear)")
                        
                        if scaling_factor < 0.8:
                            print(f"  → Sub-linear scaling (efficient)")
                        elif scaling_factor > 1.2:
                            print(f"  → Super-linear scaling (inefficient, potential bottleneck)")
                        else:
                            print(f"  → Near-linear scaling (expected)")


## 📊 KPI Impact Analysis

*Note: Full KPI analysis requires evaluation episodes and metric collection. This section can be extended to measure social welfare, market efficiency, convergence time, and coordination effectiveness vs agent count.*


## 📝 Results Summary


In [ ]:
if not df_results.empty and len(df_success) > 0:
    print("📝 Scalability Analysis Results Summary")
    print("=" * 80)
    
    # Best and worst performing combinations
    print("\n🏆 Best Performing Combinations (Fastest Training):")
    best_combos = df_success.nsmallest(5, 'training_time')[['num_agents', 'training_mode', 'algorithm', 'training_time']]
    for idx, row in best_combos.iterrows():
        print(f"  {row['num_agents']} agents, {row['training_mode']}, {row['algorithm']}: {row['training_time']:.2f}s")
    
    print("\n🐌 Worst Performing Combinations (Slowest Training):")
    worst_combos = df_success.nlargest(5, 'training_time')[['num_agents', 'training_mode', 'algorithm', 'training_time']]
    for idx, row in worst_combos.iterrows():
        print(f"  {row['num_agents']} agents, {row['training_mode']}, {row['algorithm']}: {row['training_time']:.2f}s")
    
    # Recommendations
    print("\n💡 Key Findings:")
    
    # Best training mode for scalability
    mode_avg = df_success.groupby('training_mode')['training_time'].mean().sort_values()
    print(f"\n  Best Training Mode (avg time): {mode_avg.index[0]} ({mode_avg.iloc[0]:.2f}s)")
    
    # Best algorithm for scalability
    algo_avg = df_success.groupby('algorithm')['training_time'].mean().sort_values()
    print(f"  Best Algorithm (avg time): {algo_avg.index[0]} ({algo_avg.iloc[0]:.2f}s)")
    
    # Scalability limits
    max_agents_tested = df_success['num_agents'].max()
    max_time = df_success['training_time'].max()
    print(f"\n  Maximum Agents Tested: {max_agents_tested}")
    print(f"  Maximum Training Time: {max_time:.2f}s ({max_time/60:.1f} minutes)")
    
    print("\n📁 Results saved to:")
    print(f"  - {ScalabilityConfig.RESULTS_DIR / 'scalability_results_final.csv'}")
    print(f"  - {ScalabilityConfig.RESULTS_DIR / 'computational_time_analysis.png'}")
    
    print("\n✅ Analysis complete!")
